datasets :

https://www.kaggle.com/datasets/loveall/appliances-energy-prediction *household data.

https://www.kaggle.com/datasets/csafrit2/steel-industry-energy-consumption * classification problem, industry data

* both data has been filtered through 9 kaggle pages. change field if not viable.


https://www.kaggle.com/code/winniewg05/classification-on-load-type

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import joblib
from google.colab import files
import pandas as pd

In [ ]:
print(joblib.__version__)

1.4.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/# MY JOBS/TensorLane - Big Data Solution Architect/2. Enhancing operational efficiency and agility/Steel_industry_data.csv')

# EDA

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35040 entries, 0 to 35039
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   date                                  35040 non-null  object 
 1   Usage_kWh                             35040 non-null  float64
 2   Lagging_Current_Reactive.Power_kVarh  35040 non-null  float64
 3   Leading_Current_Reactive_Power_kVarh  35040 non-null  float64
 4   CO2(tCO2)                             35040 non-null  float64
 5   Lagging_Current_Power_Factor          35040 non-null  float64
 6   Leading_Current_Power_Factor          35040 non-null  float64
 7   NSM                                   35040 non-null  int64  
 8   WeekStatus                            35040 non-null  object 
 9   Day_of_week                           35040 non-null  object 
 10  Load_Type                             35040 non-null  object 
dtypes: float64(6), 

In [ ]:
# renaming column
df = df.rename(columns={'Lagging_Current_Reactive.Power_kVarh': 'Lagging_Reactive_Power_kVarh',
                        'Leading_Current_Reactive_Power_kVarh': 'Leading_Reactive_Power_kVarh',
                        'Lagging_Current_Power_Factor': 'Lagging_Power_Factor',
                        'Leading_Current_Power_Factor': 'Leading_Power_Factor',
                        'CO2(tCO2)':'CO2'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35040 entries, 0 to 35039
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   date                          35040 non-null  object 
 1   Usage_kWh                     35040 non-null  float64
 2   Lagging_Reactive_Power_kVarh  35040 non-null  float64
 3   Leading_Reactive_Power_kVarh  35040 non-null  float64
 4   CO2                           35040 non-null  float64
 5   Lagging_Power_Factor          35040 non-null  float64
 6   Leading_Power_Factor          35040 non-null  float64
 7   NSM                           35040 non-null  int64  
 8   WeekStatus                    35040 non-null  object 
 9   Day_of_week                   35040 non-null  object 
 10  Load_Type                     35040 non-null  object 
dtypes: float64(6), int64(1), object(4)
memory usage: 2.9+ MB


In [ ]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13824 entries, 0 to 33215
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   date                          13824 non-null  datetime64[ns]
 1   Usage_kWh                     13824 non-null  float64       
 2   Lagging_Reactive_Power_kVarh  13824 non-null  float64       
 3   Leading_Reactive_Power_kVarh  13824 non-null  float64       
 4   CO2                           13824 non-null  float64       
 5   Lagging_Power_Factor          13824 non-null  float64       
 6   Leading_Power_Factor          13824 non-null  float64       
 7   NSM                           13824 non-null  int64         
 8   WeekStatus                    13824 non-null  object        
 9   Day_of_week                   13824 non-null  object        
 10  Load_Type                     13824 non-null  object        
dtypes: datetime64[ns](1), float64(6),

# PREPROCESSING

In [ ]:
# categorical feature encoding
# Encode Categorical Columns
categ = df.select_dtypes(include = "object").columns

le = preprocessing.LabelEncoder()
df[categ] = df[categ].apply(le.fit_transform)

df.head()

,date,Usage_kWh,Lagging_Reactive_Power_kVarh,Leading_Reactive_Power_kVarh,CO2,Lagging_Power_Factor,Leading_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,2018-01-01 00:15:00,3.17,2.95,0.0,0.0,73.21,100.0,900,0,1,0
1,2018-01-01 00:30:00,4.00,4.46,0.0,0.0,66.77,100.0,1800,0,1,0
2,2018-01-01 00:45:00,3.24,3.28,0.0,0.0,70.28,100.0,2700,0,1,0
3,2018-01-01 01:00:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,0,1,0
4,2018-01-01 01:15:00,3.82,4.50,0.0,0.0,64.72,100.0,4500,0,1,0


In [ ]:
# Sort the dataframe by the date column
df = df.sort_values(by='date').reset_index(drop=True)

# Define the sizes for each section
size_main = 12324
size_others = 500
num_others = 3

# Split the dataframe into sections
df_1 = df.iloc[:size_main]
df_others = [df.iloc[size_main + i*size_others : size_main + (i+1)*size_others] for i in range(num_others)]

# Unpack the list into individual dataframes
df_2, df_3, df_4 = df_others

# Verify the shapes of the resulting dataframes
print(f'df_1 shape: {df_1.shape}')
for i, df_section in enumerate(df_others, 2):
    print(f'df_{i} shape: {df_section.shape}')

df_1 shape: (12324, 11)
df_2 shape: (500, 11)
df_3 shape: (500, 11)
df_4 shape: (500, 11)


In [ ]:
# download all dataframes
# Save each DataFrame as a CSV file
df_1.to_csv('df_1.csv', index=False)
df_2.to_csv('df_2.csv', index=False)
df_3.to_csv('df_3.csv', index=False)
df_4.to_csv('df_4.csv', index=False)

# Download each CSV file
files.download('df_1.csv')
files.download('df_2.csv')
files.download('df_3.csv')
files.download('df_4.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#drop date as it has low correlation
df_1 = df_1.drop('date', axis='columns')

# Split the dataset and prepare some lists to store the models
X = df_1.drop(['Load_Type'], axis=1)
y = df_1.Load_Type
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42)

### Creating the columns transformers

Training vs. Inference Consistency: A ColumnTransformer helps ensure that the data preprocessing steps applied during training are consistently applied during inference. This is crucial for maintaining the performance of the model when deployed in production.

In [ ]:
#Get the columns name
df_1.drop(columns=["Load_Type"]).select_dtypes('number').columns.tolist()

['Usage_kWh',
 'Lagging_Reactive_Power_kVarh',
 'Leading_Reactive_Power_kVarh',
 'CO2',
 'Lagging_Power_Factor',
 'Leading_Power_Factor',
 'NSM',
 'WeekStatus',
 'Day_of_week']

In [ ]:
#Specify Numerical and Categorical Columns for preprocessor
numerical_features = ['Usage_kWh','Lagging_Reactive_Power_kVarh','Leading_Reactive_Power_kVarh',
                      'CO2','Lagging_Power_Factor','Leading_Power_Factor','NSM']
passthrough_features = ['WeekStatus','Day_of_week']

#Prepare preprocessor
preprocessor = make_column_transformer(
(make_pipeline(StandardScaler()),numerical_features,),
("passthrough", passthrough_features),)

# MODEL DEFINITION

In [ ]:
xgb_model = XGBClassifier()

pipe = make_pipeline(preprocessor, xgb_model)

# MODEL TRAINING

In [ ]:
pipe.fit(X_train, y_train)
print(xgb_model)

score = pipe.score(X_test, y_test)
print("Model score: %.3f" %score)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)
Model score: 0.919


In [ ]:
print(pipe)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Usage_kWh',
                                                   'Lagging_Reactive_Power_kVarh',
                                                   'Leading_Reactive_Power_kVarh',
                                                   'CO2',
                                                   'Lagging_Power_Factor',
                                                   'Leading_Power_Factor',
                                                   'NSM']),
                                                 ('passthrough', 'passthrough',
                                                  ['WeekStatus',
                                                   'Day_of_week'])])),
             

# MODEL EVALUATION

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
#Build a text report table showing the main classification metrics and its accuracy

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      1635
           1       0.83      0.84      0.84       619
           2       0.86      0.84      0.85       827

    accuracy                           0.92      3081
   macro avg       0.89      0.89      0.89      3081
weighted avg       0.92      0.92      0.92      3081



### Making Prediction on non processed model

In [ ]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 12324 to 12823
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Usage_kWh                     500 non-null    float64
 1   Lagging_Reactive_Power_kVarh  500 non-null    float64
 2   Leading_Reactive_Power_kVarh  500 non-null    float64
 3   CO2                           500 non-null    float64
 4   Lagging_Power_Factor          500 non-null    float64
 5   Leading_Power_Factor          500 non-null    float64
 6   NSM                           500 non-null    int64  
 7   WeekStatus                    500 non-null    int64  
 8   Day_of_week                   500 non-null    int64  
dtypes: float64(6), int64(3)
memory usage: 35.3 KB


In [ ]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 12824 to 13323
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   date                          500 non-null    datetime64[ns]
 1   Usage_kWh                     500 non-null    float64       
 2   Lagging_Reactive_Power_kVarh  500 non-null    float64       
 3   Leading_Reactive_Power_kVarh  500 non-null    float64       
 4   CO2                           500 non-null    float64       
 5   Lagging_Power_Factor          500 non-null    float64       
 6   Leading_Power_Factor          500 non-null    float64       
 7   NSM                           500 non-null    int64         
 8   WeekStatus                    500 non-null    int64         
 9   Day_of_week                   500 non-null    int64         
 10  Load_Type                     500 non-null    int64         
dtypes: datetime64[ns](1), floa

In [ ]:
df_2.head(3)

,Usage_kWh,Lagging_Reactive_Power_kVarh,Leading_Reactive_Power_kVarh,CO2,Lagging_Power_Factor,Leading_Power_Factor,NSM,WeekStatus,Day_of_week
12324,93.92,54.36,0.0,0.04,86.55,100.0,32400,0,5
12325,115.13,61.56,0.0,0.05,88.19,100.0,33300,0,5
12326,106.70,69.62,0.0,0.05,83.75,100.0,34200,0,5


In [ ]:
df_3.head(3)

,date,Usage_kWh,Lagging_Reactive_Power_kVarh,Leading_Reactive_Power_kVarh,CO2,Lagging_Power_Factor,Leading_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
12824,2018-12-02 14:00:00,104.69,46.94,0.0,0.05,91.25,100.0,50400,0,1,2
12825,2018-12-02 14:15:00,65.92,22.00,0.0,0.03,94.86,100.0,51300,0,1,2
12826,2018-12-02 14:30:00,55.91,13.61,0.0,0.03,97.16,100.0,52200,0,1,2


In [ ]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 12824 to 13323
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   date                          500 non-null    datetime64[ns]
 1   Usage_kWh                     500 non-null    float64       
 2   Lagging_Reactive_Power_kVarh  500 non-null    float64       
 3   Leading_Reactive_Power_kVarh  500 non-null    float64       
 4   CO2                           500 non-null    float64       
 5   Lagging_Power_Factor          500 non-null    float64       
 6   Leading_Power_Factor          500 non-null    float64       
 7   NSM                           500 non-null    int64         
 8   WeekStatus                    500 non-null    int64         
 9   Day_of_week                   500 non-null    int64         
 10  Load_Type                     500 non-null    int64         
dtypes: datetime64[ns](1), floa

In [ ]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 13324 to 13823
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Usage_kWh                     500 non-null    float64
 1   Lagging_Reactive_Power_kVarh  500 non-null    float64
 2   Leading_Reactive_Power_kVarh  500 non-null    float64
 3   CO2                           500 non-null    float64
 4   Lagging_Power_Factor          500 non-null    float64
 5   Leading_Power_Factor          500 non-null    float64
 6   NSM                           500 non-null    int64  
 7   WeekStatus                    500 non-null    int64  
 8   Day_of_week                   500 non-null    int64  
dtypes: float64(6), int64(3)
memory usage: 35.3 KB


In [ ]:
y_true = df_4['Load_Type']

df_4 = df_4.drop(['date', 'Load_Type'], axis='columns')

In [ ]:
score = pipe.score(df_4, y_true)
print("Model score: %.3f" %score)

Model score: 0.896


In [ ]:
y_pred = pipe.predict(df_4)
y_pred[:33]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
y_pred = pipe.predict(df_2)
y_pred[:33]

array([0, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
y_pred = model.predict(df_2)
y_pred[:33]

array([0, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
ypred_tolist = y_pred.tolist()

# INVOCATION TEST

In [ ]:
# model is working and tested
model = joblib.load("xgboost_model.joblib")

In [ ]:
def predict(body, model) :
    """
    Generate predictions for the incoming request using the model.
    """
    features = pd.DataFrame.from_records(body["features"])
    predictions = model.predict(features)
    return {"predictions": predictions}

In [ ]:
X = df_2.sample(33)
# X = X.drop(['date', 'Load_Type'], axis='columns')
features = X.to_dict(orient='records')

features_dict = {"features": features}

In [ ]:
predictions = predict(features_dict, model)

In [ ]:
predictions

{'predictions': array([0, 0, 0, 0, 2, 0, 1, 1, 1, 2, 1, 0, 1, 2, 0, 2, 2, 0, 1, 0, 0, 1,
        1, 0, 0, 0, 2, 0, 0, 0, 1, 2, 2])}

In [ ]:
import sklearn
print(sklearn.__version__)


1.3.2


# SAVING MODEL

In [ ]:
# Save the entire pipeline in .joblib format
joblib.dump(pipe, 'xgb_model.joblib')

['xgb_model.joblib']